In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
DATADIR = os.getenv('DATADIR')
DATADIR

'/data/2018-03-27'

In [4]:
top5000 = pd.read_csv(
    os.path.join(DATADIR, 'govuk_top5000_pageviews.csv'),
    dtype=object
)

In [12]:
top5000

,Page,Page Views,Unique Page Views,Avg. Time on Page,Entrances,Bounce Rate,% Exit,Page Value
0,/vehicle-tax,"5,085,364","4,153,337",00:02:49,"3,278,792",8.75%,67.31%,£0.00
1,/sold-bought-vehicle,"3,698,926","826,906",00:00:36,"483,227",11.13%,17.23%,£0.00
2,/government/organisations/companies-house,"3,238,486","2,589,493",00:00:23,"2,392,275",6.17%,9.23%,£0.00
3,/check-vehicle-tax,"3,064,645","2,412,981",00:02:41,"1,788,035",6.88%,68.26%,£0.00
4,/get-information-about-a-company,"2,961,689","2,516,742",00:06:39,"511,605",5.68%,77.24%,£0.00
5,/check-mot-history,"2,904,878","2,305,258",00:04:06,"1,907,321",6.39%,70.52%,£0.00
6,/log-in-register-hmrc-online-services,"2,689,168","2,216,366",00:06:05,"1,081,591",4.30%,66.75%,£0.00
7,/calculate-your-child-maintenance,"2,683,322","279,437",00:00:25,"237,690",22.93%,9.01%,£0.00
8,/state-pension-age,"2,569,230","414,732",00:00:23,"279,041",14.84%,9.55%,£0.00
9,/calculate-your-holiday-entitlement,"2,413,923","267,341",00:00:29,"171,191",17.38%,9.25%,£0.00


In [6]:
predictions_uk = pd.read_csv(
    os.path.join(DATADIR, 'predictions_noukeconomy.csv.gz'),
    dtype=object,
    compression='gzip'
)

In [38]:
predictions_uk.shape

(88725, 17)

In [31]:
predictions_uk_unique_content = predictions_uk.drop_duplicates('content_id')

In [39]:
predictions_uk_unique_content.shape

(64438, 17)

In [37]:
predictions_uk[['base_path', 'title', 'taxon2label']].to_csv(
    os.path.join(DATADIR, 'prediction_uk_clean.csv.gz'),compression='gzip',index=False)

In [32]:
merged = pd.merge(
    left=predictions_uk_unique_content,
    right=top5000,
    left_on='base_path',
    right_on='Page',
    how='outer',
    indicator=True
)

In [33]:
merged.groupby('_merge').size()

_merge
left_only     64107
right_only     4699
both            331
dtype: int64

In [10]:
merged[merged['_merge']=='both']

,content_id,prob,taxon2label,base_path,description,document_type,document_type_gp,first_published_at,level1taxon,locale,...,prob_cat,Page,Page Views,Unique Page Views,Avg. Time on Page,Entrances,Bounce Rate,% Exit,Page Value,_merge
20,58277f0b-ba8a-488f-8332-4e66c6f39ecc,0.5120241045951843,Administrative justice reform,/government/news/hm-courts-tribunals-service-a...,hmcts provides advice to debtors who may recei...,news_story,news_and_announcements,2017-02-01 15:31:00.000,"Crime, justice and law",en,...,0.5-0.59,/government/news/hm-courts-tribunals-service-a...,"4,664","3,897",00:04:01,"2,981",69.13%,63.23%,£0.00,both
5911,fbcff459-0701-4f8f-a8cb-19a952275ee7,0.9992703795433044,Arts and culture,/government/news/culture-is-digital-report-wil...,museums galleries theatres and heritage organi...,press_release,news_and_announcements,2018-03-07 09:00:18.000,NaN,en,...,>=0.8,/government/news/culture-is-digital-report-wil...,"5,101","4,477",00:02:26,"3,427",62.30%,54.93%,£0.00,both
6157,1f576178-5097-4672-b48f-fd92a2f1e7a3,0.9110828638076782,Arts and culture,/government/publications/culture-is-digital,using technology to drive audience engagement ...,policy_paper,other,2018-03-07 09:00:00.000,NaN,en,...,>=0.8,/government/publications/culture-is-digital,"6,651","5,170",00:03:04,"3,478",37.12%,59.60%,£0.00,both
6158,1f576178-5097-4672-b48f-fd92a2f1e7a3,0.6722391843795776,Media and communications,/government/publications/culture-is-digital,using technology to drive audience engagement ...,policy_paper,other,2018-03-07 09:00:00.000,NaN,en,...,0.6-0.69,/government/publications/culture-is-digital,"6,651","5,170",00:03:04,"3,478",37.12%,59.60%,£0.00,both
7295,a35a6a63-8865-4ea3-8b50-db14c55da6a1,1.0,Brexit,/government/news/david-davis-statement-eu-uk-a...,statement in brussels by david davis secretary...,press_release,news_and_announcements,2018-03-19 13:11:12.000,NaN,en,...,>=0.8,/government/news/david-davis-statement-eu-uk-a...,"5,908","5,386",00:03:49,"3,504",84.65%,67.92%,£0.00,both
7296,a35a6a63-8865-4ea3-8b50-db14c55da6a1,1.0,Europe [PA] (Level 2 Government theme),/government/news/david-davis-statement-eu-uk-a...,statement in brussels by david davis secretary...,press_release,news_and_announcements,2018-03-19 13:11:12.000,NaN,en,...,>=0.8,/government/news/david-davis-statement-eu-uk-a...,"5,908","5,386",00:03:49,"3,504",84.65%,67.92%,£0.00,both
7354,3bdf7197-5cd8-44b9-8dd3-20881bc86c05,0.9787369966506958,Brexit,/government/publications/draft-withdrawal-agre...,marking progress by the negotiators of the eur...,policy_paper,other,2018-03-19 11:52:50.000,NaN,en,...,>=0.8,/government/publications/draft-withdrawal-agre...,"22,055","19,321",00:06:11,"17,209",32.19%,81.21%,£0.00,both
7355,3bdf7197-5cd8-44b9-8dd3-20881bc86c05,0.9250683784484864,Europe [PA] (Level 2 Government theme),/government/publications/draft-withdrawal-agre...,marking progress by the negotiators of the eur...,policy_paper,other,2018-03-19 11:52:50.000,NaN,en,...,>=0.8,/government/publications/draft-withdrawal-agre...,"22,055","19,321",00:06:11,"17,209",32.19%,81.21%,£0.00,both
7366,e36042a5-b0e9-4d74-bdbc-f922fc2c9aa1,0.8536098003387451,Brexit,/government/publications/eu-citizens-arriving-...,the government’s offer for eu citizens and the...,policy_paper,other,2018-02-28 14:24:49.000,NaN,en,...,>=0.8,/government/publications/eu-citizens-arriving-...,"8,540","6,649",00:01:31,"3,406",35.93%,34.80%,£0.00,both
7367,e36042a5-b0e9-4d74-bdbc-f922fc2c9aa1,0.9971964359283448,Europe [PA] (Level 2 Government theme),/government/publications/eu-citizens-arriving-...,the government’s offer for eu citizens and the...,policy_paper,other,2018-02-28 14:24:49.000,NaN,en,...,>=0.8,/government/publications/eu-citizens-arriving-...,"8,540","6,649",00:01:31,"3,406",35.93%,34.80%,£0.00,both


In [11]:
clean_content = pd.read_csv(
    os.path.join(DATADIR, 'clean_content.csv.gz'),
    dtype=object,
    compression='gzip'
)

In [13]:
merged2 = pd.merge(
    left=clean_content,
    right=top5000,
    left_on='base_path',
    right_on='Page',
    how='outer',
    indicator=True
)

In [15]:
merged2.groupby('_merge').size()

_merge
left_only     212096
right_only      2895
both            3802
dtype: int64

In [16]:
485/3802

0.12756443976854287

In [21]:
merged2[merged2['_merge']=='both'].groupby('document_type').size()

document_type
answer                   525
calendar                   3
case_study                 1
consultation_outcome       7
correspondence             4
decision                   3
detailed_guide           540
document_collection      216
drug_safety_update         1
foi_release                1
form                     263
government_response        1
guidance                 404
guide                   1025
independent_report         2
international_treaty       2
licence                    9
local_transaction        117
manual                    12
map                        4
national_statistics        7
news_story                49
notice                    20
official_statistics        7
place                     18
policy                     1
policy_paper              28
press_release             11
promotional                8
recruitment                1
regulation                 2
research                   3
simple_smart_answer       30
special_route              4


In [22]:
merged[merged['_merge']=='both'].groupby('document_type').size()

document_type
answer                  22
closed_consultation      2
correspondence           3
detailed_guide          79
document_collection     35
form                    34
guidance                66
guide                   51
hmrc_manual              6
independent_report       7
licence                  9
manual                   9
medical_safety_alert     5
national_statistics      3
news_story              35
notice                   1
official_statistics      6
open_consultation       10
oral_statement           3
policy                   4
policy_paper            21
press_release           23
promotional              2
recruitment             14
regulation               1
research                 2
simple_smart_answer      2
speech                   3
statistics               3
statutory_guidance       2
take_part                1
transaction             18
transparency             1
written_statement        2
dtype: int64

In [24]:
merged2[merged2['_merge']=='both'].groupby('publishing_app').size()

publishing_app
calendars                   3
collections-publisher       1
frontend                    1
manuals-publisher          12
policy-publisher            1
publisher                2047
smartanswers               63
specialist-publisher        1
whitehall                1673
dtype: int64

In [25]:
merged[merged['_merge']=='both'].groupby('publishing_app').size()

publishing_app
hmrc-manuals-api          6
manuals-publisher         9
policy-publisher          4
publisher                94
smartanswers              8
specialist-publisher      5
whitehall               359
dtype: int64

In [26]:
unlabelled_meta = pd.read_csv(
    os.path.join(DATADIR, 'unlabelled_predictions_meta.csv.gz'),
    dtype=object,
    compression='gzip'
)

In [28]:
unlabelled_unique_content = unlabelled_meta.drop_duplicates('content_id')

In [29]:
merged3 = pd.merge(
    left=unlabelled_unique_content,
    right=top5000,
    left_on='base_path',
    right_on='Page',
    how='outer',
    indicator=True
)

In [30]:
merged3.groupby('_merge').size()

_merge
left_only     84968
right_only     4564
both            466
dtype: int64

In [34]:
331/466

0.7103004291845494